In [1]:
import os
from glob import glob
from natsort import natsorted
import numpy as np
import subprocess
import nibabel as nib
import json
from tqdm import tqdm
import pandas as pd

In [2]:
# define the target image for the registration and the corresponding face mask
target = "./data/mean_reg2mean.nii.gz"
target_face_mask = "./data/facemask.nii.gz"

# define the list iof images that need to be defaced
floating_imgs = natsorted(glob("./data/example_input_images/*/*.nii.gz"))

# define a list of existing transforms (for example created with 3D Slicer) that are used instead of the automatic registration
# value should be None if no transform exists and automatic registration is to be used
existing_transform_paths = [os.path.join(os.path.dirname(f), "Transform_to_template.txt") for f in floating_imgs]
existing_transform_paths = [f if os.path.isfile(f) else "None" for f in existing_transform_paths ]

# list of output paths for each defaced image
results_folder_paths = [os.path.join("./data/example_output/defaced_images", *f.split(os.sep)[-2:-1]) for f in floating_imgs]

# check that all lists have the same length
assert len(floating_imgs) == len(results_folder_paths) == len(existing_transform_paths), f"The lists have different lengths, {len(floating_imgs)=}, {len(results_folder_paths)=}, {len(existing_transform_paths)=}"

print(f"Found {len(floating_imgs)} images that need to be defaced")

# print example paths
print("Input images: ", floating_imgs[0:3])
print("Existing transforms: ", existing_transform_paths[0:3])
print("Output paths: ", results_folder_paths[0:3])

Found 2 images that need to be defaced
Input images:  ['./data/example_input_images/IXI002-Guys-0828-T1/IXI002-Guys-0828-T1.nii.gz', './data/example_input_images/IXI002-Guys-0828-T2/IXI002-Guys-0828-T2.nii.gz']
Existing transforms:  ['None', './data/example_input_images/IXI002-Guys-0828-T2/Transform_to_template.txt']
Output paths:  ['./data/example_output/defaced_images/IXI002-Guys-0828-T1', './data/example_output/defaced_images/IXI002-Guys-0828-T2']


In [3]:
for i, (floating, results_folder_path, ex_tfm) in enumerate(tqdm(list(zip(floating_imgs, results_folder_paths, existing_transform_paths)))):
    print(i)

    if os.path.isfile(os.path.join(results_folder_path, os.path.basename(floating).replace(".nii.gz", "_masked.nii.gz"))):
        print("file exists. skipping ..... ")
        continue
    else:
        os.makedirs(results_folder_path, exist_ok=True)

        out_affine = os.path.join(results_folder_path, os.path.basename(floating.replace("nii.gz", "txt")))
        out_resampled_img_path = os.path.join(results_folder_path, os.path.basename(floating).replace(".nii.gz", "_resampled.nii.gz"))
        
        target_nii = nib.load(target)
        floating_nii = nib.load(floating)
        
        # path to BRAINSFit executable, or simply "BRAINSFit" if it is in the PATH
        BRAINSFit_bin_path = os.path.join("./BRAINSTools/BRAINSFit")
        
        if os.path.isfile(ex_tfm):
            print(f"Using existing initialTransform {ex_tfm} instead of running registration...")
            out_affine = ex_tfm
        else:
            print(f"Run registration with BRAINSFit...")
            os.system(      f' "{BRAINSFit_bin_path}" ' +
                            f'--fixedVolume "{target}" ' + 
                            f'--movingVolume "{floating}" ' + 
                            f'--outputVolume "{out_resampled_img_path}" ' + 
                            f'--outputTransform "{out_affine}" ' + 
                            f'--samplingPercentage 0.1  ' +  # reduce this for speed up, but maybe less accurate
                            f'--splineGridSize 14,10,12  ' + 
                            f'--initializeTransformMode useMomentsAlign  ' +  # Off|useMomentsAlign|useCenterOfHeadAlign|useGeometryAlign|useCenterOfROIAlign
                            f'--useRigid  ' + 
                            f'--useAffine  ' + 
                            f'--maskProcessingMode NOMASK  ' + 
                            f'--medianFilterSize 0,0,0  ' + 
                            f'--removeIntensityOutliers 0  ' + 
                            f'--outputVolumePixelType float  ' + 
                            f'--backgroundFillValue 0  ' + 
                            f'--interpolationMode Linear  ' + 
                            f'--numberOfIterations 1500  ' + 
                            f'--maximumStepLength 0.05  ' + 
                            f'--minimumStepLength 0.001  ' + 
                            f'--relaxationFactor 0.5  ' + 
                            f'--translationScale 1000  ' + 
                            f'--reproportionScale 1  ' + 
                            f'--skewScale 1  ' + 
                            f'--maxBSplineDisplacement 0  ' + 
                            f'--fixedVolumeTimeIndex 0  ' + 
                            f'--movingVolumeTimeIndex 0  ' + 
                            f'--numberOfHistogramBins 50  ' + 
                            f'--numberOfMatchPoints 10  ' + 
                            f'--costMetric MMI  ' + 
                            f'--maskInferiorCutOffFromCenter 1000  ' + 
                            f'--ROIAutoDilateSize 0  ' + 
                            f'--ROIAutoClosingSize 9  ' + 
                            f'--numberOfSamples 0  ' + 
                            f'--failureExitCode -1  ' + 
                            f'--numberOfThreads -1  ' + 
                            f'--debugLevel 0  ' + 
                            f'--costFunctionConvergenceFactor 2e+13  ' + 
                            f'--projectedGradientTolerance 1e-05  ' + 
                            f'--maximumNumberOfEvaluations 900  ' + 
                            f'--maximumNumberOfCorrections 25  ' + 
                            f'--metricSamplingStrategy Random '
                            f'>> /dev/null'  # remove this to see BRAINSFit output
                    )
        
        # transform face mask to space of the floating image
        os.system(f'./apply_affine "{target_face_mask}" "{floating}" "{out_affine}" "{results_folder_path}" -noshow')
        
        # define path to the registered and resampled face mask
        face_mask_resampled_path = os.path.join(results_folder_path, os.path.basename(target_face_mask).replace(".nii.gz", "_resampled.nii.gz"))
        
        # path to BRAINSFit executable, or simply "BRAINSFit" if it is in the PATH
        BRAINSresample_bin_path = "./BRAINSTools/BRAINSResample"
        os.system(      f' "{BRAINSresample_bin_path}" '  + 
                        f'--inputVolume "{target_face_mask}" ' + 
                        f' --referenceVolume "{floating}" ' +
                        f'--outputVolume "{face_mask_resampled_path}" ' + 
                        f'--warpTransform "{out_affine}" ' + 
                        f'--inverseTransform ' +
                        f'--interpolationMode NearestNeighbor '
                        f'>> /dev/null'
                )

        # apply mask to image
        floating_masked_path = os.path.join(results_folder_path, os.path.basename(floating).replace(".nii.gz", "_masked.nii.gz"))

        floating_nii = nib.load(floating)
        floating_data = floating_nii.get_fdata()

        mask_nii = nib.load(face_mask_resampled_path)
        mask_data = mask_nii.get_fdata()

        floating_masked_data = floating_data * mask_data
        floating_masked_nii = nib.Nifti1Image(floating_masked_data, affine = floating_nii.affine)

        nib.save(floating_masked_nii, floating_masked_path)

  0%|          | 0/2 [00:00<?, ?it/s]

0
Run registration with BRAINSFit...
Original Fixed image origin[100.006, 113.158, -121.89, 0]
TransformTypes: Rigid(1 of 2).

TransformTypes: Affine(2 of 2).

Initializing transform with useMomentsAlign
Initializing transform with useMomentsAlign to 
VersorRigid3DTransform (0x556ad4f813b0)
  RTTI typeinfo:   itk::VersorRigid3DTransform<double>
  Reference Count: 2
  Modified Time: 799
  Debug: Off
  Object Name: 
  Observers: 
    none
  Matrix: 
    1 0 0 
    0 1 0 
    0 0 1 
  Offset: [-7.57824, 3.64726, 29.4162]
  Center: [6.46315, -1.61515, -23.3726]
  Translation: [-7.57824, 3.64726, 29.4162]
  Inverse: 
    1 0 0 
    0 1 0 
    0 0 1 
  Singular: 0
  Versor: [ 0, 0, 0, 1 ]




=============================== ITKv4 Registration: Starting Transform Estimations for Rigid(1 of 2).===============================
Stop condition from optimizer.RegularStepGradientDescentOptimizerv4: Step too small after 126 iterations. Current step (0.00078125) is less than minimum step (0.001).



=

double free or corruption (!prev)
Aborted (core dumped)


ref_img_nii:
---
qform code = 1
[[   0.           0.           1.1999969  -88.6398926]
 [  -0.9303523    0.1155457    0.         116.5320053]
 [   0.1155457    0.9303523   -0.        -112.1135559]
 [   0.           0.           0.           1.       ]]
sform code = 1
[[   0.           0.           1.1999969  -88.6398926]
 [  -0.9303523    0.1155457    0.         116.5320053]
 [   0.1155457    0.9303523   -0.        -112.1135559]
 [   0.           0.           0.           1.       ]]
-------------------
flo_img_nii:
---
qform code = 1
[[   0.9985747    0.0511093    0.0102124 -100.0058975]
 [  -0.0523331    0.9752242   -0.0005352 -113.1583786]
 [  -0.0104718    0.           0.976509  -121.8897018]
 [   0.           0.           0.           1.       ]]
sform code = 1
[[   0.9985747    0.0511093    0.0102124 -100.0058975]
 [  -0.0523331    0.9752241   -0.0005352 -113.1583786]
 [  -0.0104718    0.           0.976509  -121.8897018]
 [   0.           0.           0.           1.       ]]
--

 50%|█████     | 1/2 [00:51<00:51, 51.95s/it]

1
Using existing initialTransform ./data/example_input_images/IXI002-Guys-0828-T2/Transform_to_template.txt instead of running registration...
ref_img_nii:
---
qform code = 1
[[  -0.9375      -0.           0.         120.7598801]
 [  -0.           0.9317118   -0.1386887 -104.1710052]
 [  -0.           0.1040165    1.2422824  -60.286499 ]
 [   0.           0.           0.           1.       ]]
sform code = 1
[[  -0.9375      -0.          -0.         120.7598801]
 [  -0.           0.9317117   -0.1386887 -104.1710052]
 [  -0.           0.1040165    1.2422824  -60.286499 ]
 [   0.           0.           0.           1.       ]]
-------------------
flo_img_nii:
---
qform code = 1
[[   0.9985747    0.0511093    0.0102124 -100.0058975]
 [  -0.0523331    0.9752242   -0.0005352 -113.1583786]
 [  -0.0104718    0.           0.976509  -121.8897018]
 [   0.           0.           0.           1.       ]]
sform code = 1
[[   0.9985747    0.0511093    0.0102124 -100.0058975]
 [  -0.0523331    0.97522

100%|██████████| 2/2 [00:55<00:00, 27.78s/it]
